In [ ]:
import pandas as pd
import csv as csv
import pandera as pa

### LIMPEZA E LEITURA DOS DADOS

In [ ]:
pd.set_option('display.max_rows', 5752)
valores_ausentes=['**', '***','****','*****','###!', '####', 'NULL']
df = pd.read_csv("ocorrencia_2010_2020_alterado.csv", sep=';', encoding="cp1252", parse_dates=["ocorrencia_dia"], dayfirst=True, na_values=valores_ausentes)
df.head(10)


### VALIDAÇÃO DOS DADOS


In [ ]:
schema= pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo":pa.Column(pa.String, nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$') ,nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
               
    }
)

In [ ]:
schema.validate(df)

In [ ]:
df.dtypes

##### Localizando por label (loc) ou por index (iloc)

In [ ]:
df.loc[1]

In [ ]:
#Com o index (função iloc), há um range de dados em uma lista, podendo usar -1 para identificar o último elemento
df.iloc[-1]

In [ ]:
df.loc[:, 'ocorrencia_uf']

In [ ]:
df['ocorrencia_uf']

In [ ]:
df.ocorrencia_uf

##### Manipulação de Valores 

In [ ]:
#Valores Nulos (na)
df.isna().sum()

In [ ]:
#Valores Nulos (null)
df.isnull().sum()

In [ ]:
#Verificando apenas em uma coluna
#Primeira forma de fazer
df.loc[df.ocorrencia_uf.isnull()]

In [ ]:
#Segunda forma de fazer
filtro=df.ocorrencia_aerodromo.isnull()
df.loc[filtro]

In [ ]:
filtro=df.ocorrencia_hora.isnull()
df.loc[filtro]

##### Contagem de valores NÃO NULOS

In [ ]:
df.count()

### Filtros/ Agregação

In [ ]:
#ocorrencias com mais de 10 recomendações
filtro=df.total_recomendacoes > 10
df.loc[filtro]

In [ ]:
#ocorrencias com mais de 10 recomendações
filtro=df.total_recomendacoes > 10
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes']]

In [ ]:
#ocorrencias cuja classificação == INCIDENTE GRAVE
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro]

In [ ]:
#ocorrencias cuja classificação == INCIDENTE GRAVE , no estado de SP
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE' 
filtro2 = df.ocorrencia_uf=='SP'
df.loc[filtro1 & filtro2]

In [ ]:
#ocorrencias cuja classificação == INCIDENTE GRAVE ou no estado de SP
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE' 
filtro2 = df.ocorrencia_uf=='SP'
df.loc[filtro1 | filtro2]

In [ ]:
#ocorrencias cuja classificação == INCIDENTE GRAVE ou INCIDENTE  , no estado de SP
filtro1 = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE')
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

In [ ]:
#Segunda forma de fazer:
#ocorrencias cuja classificação == INCIDENTE GRAVE ou INCIDENTE  , no estado de SP
filtro1 = df.ocorrencia_classificacao.isin (['INCIDENTE GRAVE','INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

In [ ]:
#ocorrencias cuja cidade comecem com a letra C
filtro= df.ocorrencia_cidade.str[0]=="C"
df.loc[filtro]

In [ ]:
#ocorrencias cuja cidade terminam com A
filtro= df.ocorrencia_cidade.str[-1]=="A"
df.loc[filtro]

In [ ]:
#ocorrencias cuja cidade terminam com MA
filtro= df.ocorrencia_cidade.str[-2:]=="MA"
df.loc[filtro]

In [ ]:
#ocorrencias cuja cidade contém em qualquer parte do conteúdo os caracteres MA
filtro= df.ocorrencia_cidade.str.contains('MA')
df.loc[filtro]

In [ ]:
#ocorrencias cuja cidade contém em qualquer parte do conteúdo os caracteres MA ou AL
filtro= df.ocorrencia_cidade.str.contains('MA| AL')
df.loc[filtro]

In [ ]:
#ocorrencias do ano de 2015
filtro= df.ocorrencia_dia.dt.year == 2015
df.loc[filtro]

In [ ]:
#ocorrencias do ano de 2015 e mês 12
filtro1= df.ocorrencia_dia.dt.year == 2015
filtro2= df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

In [ ]:
#ocorrencias do ano de 2015 e mês 12 e dia 08
filtro1= df.ocorrencia_dia.dt.year == 2015
filtro2= df.ocorrencia_dia.dt.month == 12
filtro3= df.ocorrencia_dia.dt.day == 8
df.loc[filtro1 & filtro2 & filtro3]

In [ ]:
#ocorrencias do ano de 2015 e mês 12 e do dia 3 a 08
filtro1= df.ocorrencia_dia.dt.year == 2015
filtro2= df.ocorrencia_dia.dt.month == 12
filtro3= (df.ocorrencia_dia.dt.day > 2) & (df.ocorrencia_dia.dt.day <9)
df.loc[filtro1 & filtro2 & filtro3]

In [ ]:
#criação de nova coluna dia com hora, com a coluna dia convertida para string
df.ocorrencia_dia.astype(str) + ' '+ df.ocorrencia_hora

In [ ]:
#criação de nova coluna com conversão para Datetime 
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' '+ df.ocorrencia_hora)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
#ocorrencias do ano de 2015 e mês 12 e do dia 3 a 08, entre 11hs às 13hs.
filtro1= df.ocorrencia_dia_hora >= '2015-12-03 11:00:00'
filtro2= df.ocorrencia_dia_hora <= '2015-12-08 13:00:00'
df.loc[filtro1 & filtro2]

In [ ]:
#ocorrencias do ano de 2015 e mês 03 e atribuindo ao novo DataFrame
filtro1= df.ocorrencia_dia.dt.year == 2015
filtro2= df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro1 & filtro2]
df201503

In [ ]:
#contagem do novo DataFrame criado
df201503.count()

In [ ]:
#contagem (apenas dos valores não nulos - 21 registros) por agrupamento
df201503.groupby(['ocorrencia_classificacao']).ocorrencia_aerodromo.count()

In [ ]:
#contagem (todos os valores - nulos e não nulos - 37 registros) por agrupamento 
df201503.groupby(['ocorrencia_classificacao']).ocorrencia_aerodromo.size()

In [ ]:
#contagem (todos os valores - nulos e não nulos - 37 registros) por agrupamento e ordenação descendente 
df201503.groupby(['ocorrencia_classificacao']).ocorrencia_aerodromo.size().sort_values(ascending = False)

In [ ]:
#criação de DataFrame somente com dados de 2010 e estados do Sudeste
filtro1 = df.ocorrencia_dia.dt.year == 2010
filtro2 = df.ocorrencia_uf.isin(['SP', 'MG', 'ES', 'RJ'])
dfsudeste2010 = df.loc[filtro1 & filtro2]
dfsudeste2010

In [ ]:
dfsudeste2010.groupby(['ocorrencia_classificacao']).size()

In [ ]:
dfsudeste2010.count()

In [ ]:
dfsudeste2010.groupby(['ocorrencia_classificacao', 'ocorrencia_uf']).size()

In [ ]:
dfsudeste2010.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

In [ ]:
filtro=dfsudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2010.loc[filtro]

In [ ]:
filtro1=dfsudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
filtro2=dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro1 & filtro2]


In [ ]:
filtro1=dfsudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2010.loc[filtro1].total_recomendacoes.sum()

In [ ]:
#contagem do total de recomendações por cidade, não mostra a qtde de NA
dfsudeste2010.groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

In [ ]:
#contagem do total de recomendações por aerodromo, (dropna = false --> mostra a qtde de NAN)
dfsudeste2010.groupby(['ocorrencia_aerodromo'], dropna = False).total_recomendacoes.sum()

In [ ]:
##contagem do total de recomendações por cidade, apenas aquelas que tiveram recomendações
filtro = dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values()

In [ ]:
##contagem do total de recomendações por cidade e mês
dfsudeste2010.loc[filtro].groupby(['ocorrencia_cidade', dfsudeste2010.ocorrencia_dia.dt.month]).total_recomendacoes.sum()